In [3]:
import os
%pwd

'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC\\research'

In [4]:
os.chdir("../")
%pwd

'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) #frozen=true means i dont want to add anything else in this
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_weights: str
    params_include_top: bool

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
            params_weights = self.params.WEIGHTS,
            params_include_top = self.params.INCLUDE_TOP
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [11]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top,   
        )

        self.save_model(path = self.config.base_model_path, model = self.model)
    
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and(freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )
        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path = self.config.updated_base_model_path, model = self.full_model)


    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

In [12]:
#running pipeline

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2025-12-07 18:13:04,596: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-12-07 18:13:04,630: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-12-07 18:13:04,636: INFO: common: created directory at : artifacts]
[2025-12-07 18:13:04,638: INFO: common: created directory at : artifacts/prepare_base_model]
[2025-12-07 18:13:06,203: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-12-07 18:13:06,511: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
